In [1]:
import torch
import torchvision
from tqdm import tqdm

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
#这个函数包括了两个操作：将图片转换为张量，以及将图片进行归一化处理
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(mean=[0.5], std=[0.5])])

In [ ]:
path = 'data'  #数据集下载后保存的目录

#下载训练集和测试集
trainData = torchvision.datasets.MNIST(path, train=True, transform=transform, download=True)
testData = torchvision.datasets.MNIST(path, train=False, transform=transform)


In [ ]:
#设定每一个Batch的大小
BATCH_SIZE = 256

#构建数据集和测试集的DataLoader
trainDataLoader = torch.utils.data.DataLoader(dataset=trainData, batch_size=BATCH_SIZE, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(dataset=testData, batch_size=BATCH_SIZE)


In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = torch.nn.Sequential(
            #The size of the picture is 28x28
            torch.nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            # 激活函数ReLU 只是对特征图中的每个像素值进行逐元素操作，将负值变为 0，正值保持不变。
            torch.nn.ReLU(),
            # 池化，用于在卷积神经网络中减少特征图的空间维度，同时保留主要的特征信息。
            # 使特征图的空间尺寸缩小了一半
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            #The size of the picture is 14x14
            torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),

            #The size of the picture is 7x7
            torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),

            torch.nn.Flatten(),

            # 全连接层的作用是将输入的特征向量与一组可学习的权重矩阵相乘，再加上偏置向量，从而输出一个新的特征向量。
            # （权重矩阵是全连接层（线性层）中的一个可学习参数，它在神经网络初始化时被自动创建，并在训练过程中通过反向传播算法不断更新。)
            #1,特征提取: 
            #在神经网络的最后几层，全连接层通常用于整合前面卷积层、池化层提取的特征，以便更好地进行分类或回归等任务。
            #2,映射到目标空间: 全连接层将特征映射到任务的目标维度（如分类的类别数）。
            #例如，最后一个全连接层的输出维度通常等于分类任务中的类别数。
            torch.nn.Linear(in_features=7 * 7 * 64, out_features=128),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=128, out_features=10),
            # Softmax 函数将输入张量的每个元素映射到 [0, 1] 的范围内，使得这些元素之和为 1。
            # 换句话说，它将输入转换为一个概率分布。
            torch.nn.Softmax(dim=1)
        )

    def forward(self, input):
        output = self.model(input)
        return output

In [ ]:
net = Net()
#将模型转换到device中，并将其结构显示出来
print(net.to(device))

In [ ]:
# 交叉熵损失度量了模型预测分布与真实分布之间的距离，损失越小，说明模型的预测越接近真实标签。
lossF = torch.nn.CrossEntropyLoss()
# 优化器用来更新神经网络 net 的参数（权重和偏置），以最小化损失函数 lossF。
optimizer = torch.optim.Adam(net.parameters())

In [ ]:
EPOCHS = 10
#存储训练过程
history = {'Test Loss': [], 'Test Accuracy': []}
for epoch in range(1, EPOCHS + 1):
    processBar = tqdm(trainDataLoader, unit='step')
    # 将模型设为训练模式，以确保某些特定层（如 Dropout、BatchNorm 等）在训练时表现正确。
    net.train(True)
    for step, (trainImgs, labels) in enumerate(processBar):
        trainImgs = trainImgs.to(device)
        labels = labels.to(device)

        # 清除上一步的梯度。
        net.zero_grad()
        # 通过前向传播计算输出
        outputs = net(trainImgs)
        loss = lossF(outputs, labels)
        # 每个样本在当前 batch 中模型预测的类别。
        predictions = torch.argmax(outputs, dim=1)
        # 模型在该 mini-batch 上预测正确的样本所占的比例。
        # labels.shape[0]是当前 mini-batch 的样本数量
        accuracy = torch.sum(predictions == labels) / labels.shape[0]
        # 进行反向传播，计算梯度。
        loss.backward()
        # 更新模型参数
        optimizer.step()
        processBar.set_description("[%d/%d] Loss: %.4f, Acc: %.4f" %
                                   (epoch, EPOCHS, loss.item(), accuracy.item()))

        # 记录模型在测试集上的表现(后续可以根据评估结果来调整训练过程)
        if step == len(processBar) - 1:
            correct, totalLoss = 0, 0
            # 将模型设置为评估模式，确保某些特定层（如 Dropout、BatchNorm 等）在测试时表现正确。
            net.train(False)
            for testImgs, labels in testDataLoader:
                testImgs = testImgs.to(device)
                labels = labels.to(device)
                outputs = net(testImgs)
                loss = lossF(outputs, labels)
                predictions = torch.argmax(outputs, dim=1)

                totalLoss += loss
                correct += torch.sum(predictions == labels)
            testAccuracy = correct / (BATCH_SIZE * len(testDataLoader))
            testLoss = totalLoss / len(testDataLoader)
            history['Test Loss'].append(testLoss.item())
            history['Test Accuracy'].append(testAccuracy.item())
            processBar.set_description("[%d/%d] Loss: %.4f, Acc: %.4f, Test Loss: %.4f, Test Acc: %.4f" %
                                       (epoch, EPOCHS, loss.item(), accuracy.item(), testLoss.item(),
                                        testAccuracy.item()))
    processBar.close()


In [ ]:
#对测试Loss进行可视化
import matplotlib.pyplot as plt

plt.plot(history['Test Loss'], label='Test Loss')
plt.legend(loc='best')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

#对测试准确率进行可视化
plt.plot(history['Test Accuracy'], color='red', label='Test Accuracy')
plt.legend(loc='best')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()
